# Building the ML dataframe

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

In [3]:
# Set max number of columns to display; default 20
pd.options.display.max_columns = 80

In [4]:
# Directory where data files will be downloaded
cwd_path = Path.cwd()
data_path = cwd_path.parent.joinpath('data')
data_push_path = cwd_path.parent.joinpath('data_to_push')

### Load in latest wind turbine data
- Note: Only 9 turbines decommissioned before 2019!

In [1218]:
# Read in latest turbine data
# Now only 30,642 after removing 3 turbines outside bounding box
df_turbines = pd.read_pickle(data_push_path / 'df_turbines_knn_blades_haversine_elevation_utc_2.pkl')
df_turbines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30638 entries, 0 to 30637
Data columns (total 30 columns):
 #   Column                                            Non-Null Count  Dtype              
---  ------                                            --------------  -----              
 0   EinheitMastrNummer                                30638 non-null  object             
 1   DatumLetzteAktualisierung                         30638 non-null  datetime64[ns, UTC]
 2   Bundesland                                        30638 non-null  object             
 3   Postleitzahl                                      30638 non-null  int64              
 4   Ort                                               30638 non-null  object             
 5   Laengengrad                                       30638 non-null  float64            
 6   Breitengrad                                       30638 non-null  float64            
 7   Registrierungsdatum                               30638 non-null  d

In [1219]:
# List of datetime column names for convenient indexing later
turbine_date_columns = [
    'DatumLetzteAktualisierung',
    'Registrierungsdatum',
    'Inbetriebnahmedatum',
    'DatumEndgueltigeStilllegung',
    'DatumBeginnVoruebergehendeStilllegung',
    'DatumWiederaufnahmeBetrieb'
]

In [371]:
# # Cast all tz naive datetime64 columns to UTC tz aware
# df_turbines[turbine_date_columns] = df_turbines[turbine_date_columns].apply(lambda series: series.dt.tz_localize('UTC'), axis=0)

# # save to pickle
# df_turbines.to_pickle(data_push_path / 'df_turbines_knn_blades_haversine_elevation_utc.pkl')

-----

# Load in all SMARD data

### Load in SMARD energy generated data (actual measured generation)
- My response variable `y` (aka target/label) 

In [1220]:
# Get all SMARD CSV files, load into dataframes, and concat them

def load_SMARD_generated_data():
    """
    Look for CSV files in the SMARD directory
    """
    list_of_dataframes = []
    for filepath in data_path.joinpath('SMARD').iterdir():
        if filepath.name.startswith('Realisierte_Erzeugung'):
            list_of_dataframes.append(pd.read_csv(filepath, sep=';'))
            
    # Check that num of rows is what I expect after concatenating dfs
    num_of_rows = 0
    for df in list_of_dataframes:
        num_of_rows += len(df)
    
    # concat dataframes
    df = pd.concat(list_of_dataframes)
    print(f'Number of rows match up: {num_of_rows == len(df)}')
    
    # Select columns to keep
    # Note: I'm using the interval start time to create the timestamps
    # ERA5 surface parameters are instantaneous so can't perfectly align anyway
    df = df[[
        'Datum',
        'Anfang',
        # 'Ende',
        'Wind Onshore [MWh] Berechnete Auflösungen'
    ]]
    
    # rename columns
    df.rename(columns={
        'Datum': 'date',
        'Anfang': 'interval_start_time', 
        # 'Ende': 'interval_end_time',
        'Wind Onshore [MWh] Berechnete Auflösungen': 'actual_generated_smard_mwh'
    }, inplace=True)
    
    # Handle dates and times to create unified datetime64 timestamps
    df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
    df['start_time_delta'] = pd.to_timedelta(df['interval_start_time'] + ':00')
    df['datetime_cet'] = df['date'] + df['start_time_delta']
    # Add CET timezone info and infer change from CET->CEST->CET, etc
    df['datetime_cet'] = df['datetime_cet'].dt.tz_localize(tz='CET', ambiguous='infer')
    
    # Drop un-needed columns
    df = df.drop(columns=['date', 'start_time_delta', 'interval_start_time'])
    # Re-arrange columns
    df = df[['datetime_cet', 'actual_generated_smard_mwh']]
    
    # Convert European thousands and decimal seperators in values to (US/UK) decimal full stop format
    translation_table = str.maketrans({'.': None, ',': '.'})
    df['actual_generated_smard_mwh'] = df['actual_generated_smard_mwh'].str.translate(translation_table).astype(float)
    
    # Sort rows by datetime_cet
    df.sort_values('datetime_cet', inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)
    
    return df.copy()

In [1221]:
# Load SMARD data
df_smard_generated = load_SMARD_generated_data()
df_smard_generated.info()

Number of rows match up: True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49655 entries, 0 to 49654
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   datetime_cet                49655 non-null  datetime64[ns, CET]
 1   actual_generated_smard_mwh  49655 non-null  float64            
dtypes: datetime64[ns, CET](1), float64(1)
memory usage: 776.0 KB


In [1222]:
df_smard_generated.isna().sum()

datetime_cet                  0
actual_generated_smard_mwh    0
dtype: int64

-----

### Load in SMARD day-ahead prices
- Wholesale prices: https://www.smard.de/page/en/wiki-article/5884/5976
- This is the day-ahead price!
    - "Weighted wholesale electricity price (day-ahead price on the exchange) for each hour [€/MWh] determined on the day-ahead auction that took place ones on the previous day - data is delivered no later than 2 hours after trading closes. Source: ENTSO-E"
- Data only goes back to 2018-10-01 (October 1, 2018)
- This is for the bidding area Germany/Luxembourg (not just Germany)

In [1223]:
# Get all SMARD CSV files, load into dataframes, and concat them

def load_SMARD_market_price_data():
    """
    Look for CSV files in the SMARD directory
    """
    list_of_dataframes = []
    for filepath in data_path.joinpath('SMARD').iterdir():
        if filepath.name.startswith('Gro_handelspreise'):
            list_of_dataframes.append(pd.read_csv(filepath, sep=';', na_values='-'))
            
    # Check that num of rows is what I expect after concatenating dfs
    num_of_rows = 0
    for df in list_of_dataframes:
        num_of_rows += len(df)
    
    # concat dataframes
    df = pd.concat(list_of_dataframes)
    print(f'Number of rows match up: {num_of_rows == len(df)}')
    
    # Select columns to keep
    # Note: I'm using the interval start time to create the timestamps
    # ERA5 surface parameters are instantaneous so can't perfectly align anyway
    df = df[[
        'Datum',
        'Anfang',
        # 'Ende',
        'Deutschland/Luxemburg [€/MWh] Originalauflösungen'
    ]]
    
    # rename columns
    df.rename(columns={
        'Datum': 'date',
        'Anfang': 'interval_start_time', 
        # 'Ende': 'interval_end_time',
        'Deutschland/Luxemburg [€/MWh] Originalauflösungen': 'day_ahead_price_eur_mwh'
    }, inplace=True)
    
    # Handle dates and times to create unified datetime64 timestamps
    df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
    df['start_time_delta'] = pd.to_timedelta(df['interval_start_time'] + ':00')
    df['datetime_cet'] = df['date'] + df['start_time_delta']
    # Add CET timezone info and infer change from CET->CEST->CET, etc
    df['datetime_cet'] = df['datetime_cet'].dt.tz_localize(tz='CET', ambiguous='infer')
    
    # Drop un-needed columns
    df = df.drop(columns=['date', 'start_time_delta', 'interval_start_time'])
    # Re-arrange columns
    df = df[['datetime_cet', 'day_ahead_price_eur_mwh']]
    
    # Convert European thousands and decimal seperators in values to (US/UK) decimal full stop format
    translation_table = str.maketrans({'.': None, ',': '.'})
    df['day_ahead_price_eur_mwh'] = df['day_ahead_price_eur_mwh'].str.translate(translation_table).astype(float)
    
    # Drop the numerous (700+) duplicate rows
    df.drop_duplicates(subset='datetime_cet', inplace=True)
    # Sort rows by datetime_cet
    df.sort_values('datetime_cet', inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)
    
    return df.copy()

In [1224]:
df_smard_market_price = load_SMARD_market_price_data()

Number of rows match up: True


In [1225]:
df_smard_market_price.isna().sum()

datetime_cet                  0
day_ahead_price_eur_mwh    6551
dtype: int64

In [1226]:
df_smard_market_price['day_ahead_price_eur_mwh'].describe()

count    43104.000000
mean        97.537291
std        106.238714
min       -500.000000
25%         35.260000
50%         55.155000
75%        118.750000
max        871.000000
Name: day_ahead_price_eur_mwh, dtype: float64

-----

### Load in SMARD installed capacity data
- Note: CSV reader infers decimal comma (European) format 54.499 as decimal point 54.499 even though it's 54,499.00!
    - Specify dtype of column in the read_csv method

In [1227]:
def load_SMARD_installed_capacity_data():
    """
    Look for CSV files in the SMARD directory
    """
    list_of_dataframes = []
    for filepath in data_path.joinpath('SMARD').iterdir():
        if filepath.name.startswith('Installierte_Erzeugungsleistung'):
            list_of_dataframes.append(
                pd.read_csv(filepath, sep=';', dtype={'Wind Onshore [MW] Berechnete Auflösungen': str})
            )
            
    # Check that num of rows is what I expect after concatenating dfs
    num_of_rows = 0
    for df in list_of_dataframes:
        num_of_rows += len(df)
    
    # concat dataframes
    df = pd.concat(list_of_dataframes)
    print(f'Number of rows match up: {num_of_rows == len(df)}')
    
    # Select columns to keep
    # Note: I'm using the interval start time to create the timestamps
    # ERA5 surface parameters are instantaneous so can't perfectly align anyway
    df = df[[
        'Datum',
        'Anfang',
        # 'Ende',
        'Wind Onshore [MW] Berechnete Auflösungen'
    ]]
    
    # rename columns
    df.rename(columns={
        'Datum': 'date',
        'Anfang': 'interval_start_time', 
        # 'Ende': 'interval_end_time',
        'Wind Onshore [MW] Berechnete Auflösungen': 'total_nominal_capacity_smard_mw'
    }, inplace=True)
    
    # Handle dates and times to create unified datetime64 timestamps
    df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
    df['start_time_delta'] = pd.to_timedelta(df['interval_start_time'] + ':00')
    df['datetime_cet'] = df['date'] + df['start_time_delta']
    # Add CET timezone info and infer change from CET->CEST->CET, etc
    df['datetime_cet'] = df['datetime_cet'].dt.tz_localize(tz='CET', ambiguous='infer')
    
    # Drop un-needed columns
    df = df.drop(columns=['date', 'start_time_delta', 'interval_start_time'])
    # Re-arrange columns
    df = df[['datetime_cet', 'total_nominal_capacity_smard_mw']]
    
    # Convert European thousands and decimal seperators in values to (US/UK) decimal full stop format
    translation_table = str.maketrans({'.': None, ',': '.'})
    df['total_nominal_capacity_smard_mw'] = df['total_nominal_capacity_smard_mw'].str.translate(translation_table).astype(float)
    
    # Sort rows by datetime_cet
    df.sort_values('datetime_cet', inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)
    
    return df.copy()

In [1228]:
df_smard_installed_capacity = load_SMARD_installed_capacity_data()

Number of rows match up: True


In [1229]:
df_smard_installed_capacity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49655 entries, 0 to 49654
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   datetime_cet                     49655 non-null  datetime64[ns, CET]
 1   total_nominal_capacity_smard_mw  49655 non-null  float64            
dtypes: datetime64[ns, CET](1), float64(1)
memory usage: 776.0 KB


In [1230]:
df_smard_installed_capacity.isna().sum()

datetime_cet                       0
total_nominal_capacity_smard_mw    0
dtype: int64

------

### Load in SMARD day-ahead generation forecast
- Wind Onshore [MWh] Berechnete Auflösungen for Germany
- Only has 24 missing values for the day of 2022-12-21 (verified on SMARD data visuals)
- TSOs transmit their forecasts to the ENTSO-E transparency platform no later than 18:00 of the previous day

In [1231]:
def load_SMARD_forecasted_generation_data():
    """
    Look for CSV files in the SMARD directory
    """
    list_of_dataframes = []
    for filepath in data_path.joinpath('SMARD').iterdir():
        if filepath.name.startswith('Prognostizierte_Erzeugung_Day-Ahead'):
            list_of_dataframes.append(pd.read_csv(filepath, sep=';', na_values='-'))
            
    # Check that num of rows is what I expect after concatenating dfs
    num_of_rows = 0
    for df in list_of_dataframes:
        num_of_rows += len(df)
    
    # concat dataframes
    df = pd.concat(list_of_dataframes)
    print(f'Number of rows match up: {num_of_rows == len(df)}')
    
    # Select columns to keep
    # Note: I'm using the interval start time to create the timestamps
    # ERA5 surface parameters are instantaneous so can't perfectly align anyway
    df = df[[
        'Datum',
        'Anfang',
        # 'Ende',
        'Wind Onshore [MWh] Berechnete Auflösungen'
    ]]
    
    # rename columns
    df.rename(columns={
        'Datum': 'date',
        'Anfang': 'interval_start_time', 
        # 'Ende': 'interval_end_time',
        'Wind Onshore [MWh] Berechnete Auflösungen': 'forecasted_generation_smard_mwh'
    }, inplace=True)
    
    # Handle dates and times to create unified datetime64 timestamps
    df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
    df['start_time_delta'] = pd.to_timedelta(df['interval_start_time'] + ':00')
    df['datetime_cet'] = df['date'] + df['start_time_delta']
    # Add CET timezone info and infer change from CET->CEST->CET, etc
    df['datetime_cet'] = df['datetime_cet'].dt.tz_localize(tz='CET', ambiguous='infer')
    
    # Drop un-needed columns
    df = df.drop(columns=['date', 'start_time_delta', 'interval_start_time'])
    # Re-arrange columns
    df = df[['datetime_cet', 'forecasted_generation_smard_mwh']]
    
    # Convert European thousands and decimal seperators in values to (US/UK) decimal full stop format
    translation_table = str.maketrans({'.': None, ',': '.'})
    df['forecasted_generation_smard_mwh'] = df['forecasted_generation_smard_mwh'].str.translate(translation_table).astype(float)
        
    # Sort rows by datetime_cet
    df.sort_values('datetime_cet', inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)
    
    return df.copy()

In [1232]:
df_smard_forecasted_generation = load_SMARD_forecasted_generation_data()

Number of rows match up: True


In [1233]:
df_smard_forecasted_generation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49655 entries, 0 to 49654
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   datetime_cet                     49655 non-null  datetime64[ns, CET]
 1   forecasted_generation_smard_mwh  49631 non-null  float64            
dtypes: datetime64[ns, CET](1), float64(1)
memory usage: 776.0 KB


In [1234]:
df_smard_forecasted_generation.isna().sum()

datetime_cet                        0
forecasted_generation_smard_mwh    24
dtype: int64

-----

-----

# Begin building ML dataframe `df_main`
- Start with one full year: 2022
    - Then bring in another year like 2021 and run through the same transformations and then concat along datetimeindex?
- Good resource on time-related feature engineering: https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html

In [1235]:
# Make time zone aware UTC?
datetime_index_utc = pd.date_range(start='2018-01-01', end='2023-08-31 23:59:59', freq='H', name='datetime_utc', tz='UTC')

# Make index to dataframe and reset
df_main = datetime_index_utc.to_frame().reset_index(drop=True)

# Extract properties and derive new columns
df_main['hour'] = df_main['datetime_utc'].dt.hour
# 0-6 (Monday-Sunday)
df_main['day_of_week'] = df_main['datetime_utc'].dt.dayofweek
df_main['day_of_month'] = df_main['datetime_utc'].dt.day
df_main['month_number'] = df_main['datetime_utc'].dt.month
df_main['year'] = df_main['datetime_utc'].dt.year

# Function to get meteorological season based on month number (1-12)
# Should I just go ahead and encode the discrete numerical value?
def get_meteorological_season(month):
    if 3 <= month <= 5:
        return 'spring'
    elif 6 <= month <= 8:
        return 'summer'
    elif 9 <= month <= 11:
        return 'autumn'
    else:
        return 'winter'
    
df_main['meteorological_season'] = df_main['month_number'].apply(get_meteorological_season)

df_main

,datetime_utc,hour,day_of_week,day_of_month,month_number,year,meteorological_season
0,2018-01-01 00:00:00+00:00,0,0,1,1,2018,winter
1,2018-01-01 01:00:00+00:00,1,0,1,1,2018,winter
2,2018-01-01 02:00:00+00:00,2,0,1,1,2018,winter
3,2018-01-01 03:00:00+00:00,3,0,1,1,2018,winter
4,2018-01-01 04:00:00+00:00,4,0,1,1,2018,winter
...,...,...,...,...,...,...,...
49651,2023-08-31 19:00:00+00:00,19,3,31,8,2023,summer
49652,2023-08-31 20:00:00+00:00,20,3,31,8,2023,summer
49653,2023-08-31 21:00:00+00:00,21,3,31,8,2023,summer
49654,2023-08-31 22:00:00+00:00,22,3,31,8,2023,summer


----

## Derive new columns `turbines_in_operation` and `total_nominal_capacity_operational_turbines_mw`

#### Function to check if turbine operational at a given UTC hour timestamp

In [1236]:
def is_operational(timestamp_utc, df_turbines):
    """
    Inputs: hourly tz aware utc timestamps, turbine dataframe
    Get a bool series of turbines that are operational at the time of the [hourly] timestamp
    """
    # Turbines that went into operation before the timestamp; returns bool series for bool indexing
    started_operations = df_turbines['Inbetriebnahmedatum'] <= timestamp_utc

    # Turbines already decommissioned before the timestamp; returns bool series for bool indexing
    already_decommissioned = df_turbines['DatumEndgueltigeStilllegung'] <= timestamp_utc

    # Turbines that went into maintenance before timestamp and haven't come back into operation before the timestamp
    # Note: Some turbines go straight from temporary maintenance to decommissioned without ever going back into operation
    still_in_maintenance_or_decommissioned = (df_turbines['DatumBeginnVoruebergehendeStilllegung'] <= timestamp_utc) & \
    ((df_turbines['DatumWiederaufnahmeBetrieb'] > timestamp_utc) | df_turbines['DatumWiederaufnahmeBetrieb'].isna())

    # Number of turbines operational at the timestamp
    # Note the tildas to inverse these bool series
    # This is a bool series with df_turbines index
    turbines_in_operation_bool_series = (started_operations & ~already_decommissioned & ~still_in_maintenance_or_decommissioned)

    return turbines_in_operation_bool_series

##### Derive column for total turbines in operation for every hourly timestamp

In [1237]:
# 1min 7s to run for 2018-23
df_main['turbines_in_operation'] = df_main['datetime_utc'].apply(lambda timestamp: is_operational(timestamp, df_turbines).sum())

##### Derive column for total nominal capacity for every hourly timestamp

In [1238]:
# Nettonennleistung is in kW; I divide by 1_000 to convert kilo-watts (kW) to mega-watts (MW)
# 1min 30s to run for 2018-23
df_main['total_nominal_capacity_operational_turbines_mw'] = df_main['datetime_utc'].apply(
    lambda timestamp: df_turbines.loc[:, 'Nettonennleistung'].loc[is_operational(timestamp, df_turbines)].sum() / 1_000)

#### Save progress to pickle

In [1240]:
# df_main.to_pickle(data_push_path / 'df_main.pkl')

In [1241]:
# df_main['turbines_in_operation'].plot()

In [1242]:
# df_main['total_nominal_capacity_operational_turbines_mw'].plot()

In [1243]:
df_main.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49656 entries, 0 to 49655
Data columns (total 9 columns):
 #   Column                                          Non-Null Count  Dtype              
---  ------                                          --------------  -----              
 0   datetime_utc                                    49656 non-null  datetime64[ns, UTC]
 1   hour                                            49656 non-null  int32              
 2   day_of_week                                     49656 non-null  int32              
 3   day_of_month                                    49656 non-null  int32              
 4   month_number                                    49656 non-null  int32              
 5   year                                            49656 non-null  int32              
 6   meteorological_season                           49656 non-null  object             
 7   turbines_in_operation                           49656 non-null  int64              
 8

-----

## Merge SMARD data onto `df_main` on key datetime
- Dataframes to merge with `df_main`:
    - `df_smard_generated`
    - `df_smard_market_price`
    - `df_smard_installed_capacity`
    - `df_smard_forecasted_generation`

##### Load in latest `df_main`

In [1244]:
df_main = pd.read_pickle(data_push_path / 'df_main.pkl')
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49656 entries, 0 to 49655
Data columns (total 9 columns):
 #   Column                                          Non-Null Count  Dtype              
---  ------                                          --------------  -----              
 0   datetime_utc                                    49656 non-null  datetime64[ns, UTC]
 1   hour                                            49656 non-null  int32              
 2   day_of_week                                     49656 non-null  int32              
 3   day_of_month                                    49656 non-null  int32              
 4   month_number                                    49656 non-null  int32              
 5   year                                            49656 non-null  int32              
 6   meteorological_season                           49656 non-null  object             
 7   turbines_in_operation                           49656 non-null  int64              
 8

##### Merge operations

In [1245]:
# Generated data; the response variable `y`
# keep the datetime_cet column for reference
df_main = df_main.merge(df_smard_generated, left_on='datetime_utc', right_on='datetime_cet', how='left')

# The day-ahead price data
# -> only add suffix to overlapping columns in right df being merged
df_main = df_main.merge(df_smard_market_price, left_on='datetime_utc', right_on='datetime_cet', how='left', suffixes=(None, '_remove'))

# Installed capacity
df_main = df_main.merge(df_smard_installed_capacity, left_on='datetime_utc', right_on='datetime_cet', how='left', suffixes=(None, '_remove'))

# The day-ahead forecasted generation
df_main = df_main.merge(df_smard_forecasted_generation, left_on='datetime_utc', right_on='datetime_cet', how='left', suffixes=(None, '_remove'))

##### Remove all the duplicate `datetime_cet` columns with suffix `_remove`

In [1246]:
# filter using regex; $ binds the expression to end of a string
df_main.drop(df_main.filter(regex='_remove$').columns, axis=1, inplace=True)

#### Save progress to pickle

In [1247]:
# df_main.to_pickle(data_push_path / 'df_main_smard.pkl')

-----

----

## Next step here...

##### Load in latest `df_main`

In [1311]:
df_main = pd.read_pickle(data_push_path / 'df_main_smard.pkl')
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49656 entries, 0 to 49655
Data columns (total 14 columns):
 #   Column                                          Non-Null Count  Dtype              
---  ------                                          --------------  -----              
 0   datetime_utc                                    49656 non-null  datetime64[ns, UTC]
 1   hour                                            49656 non-null  int32              
 2   day_of_week                                     49656 non-null  int32              
 3   day_of_month                                    49656 non-null  int32              
 4   month_number                                    49656 non-null  int32              
 5   year                                            49656 non-null  int32              
 6   meteorological_season                           49656 non-null  object             
 7   turbines_in_operation                           49656 non-null  int64              
 

#### Set datetime_utc column to index

In [1312]:
df_main.set_index('datetime_utc', inplace=True)

-------

## Merge mean hourly ERA5 data vars across all used grid points into `df_main`

#### Read in latest ERA5 dataset that I've combined, subsetted, and with the new data vars I derived

In [1313]:
ds = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_combined_subset_derived.nc'))
ds.data_vars

Data variables:
    u100                       (time, latitude, longitude) float32 ...
    v100                       (time, latitude, longitude) float32 ...
    u10                        (time, latitude, longitude) float32 ...
    v10                        (time, latitude, longitude) float32 ...
    t2m                        (time, latitude, longitude) float32 ...
    i10fg                      (time, latitude, longitude) float32 ...
    msl                        (time, latitude, longitude) float32 ...
    mean_wind_speed_10m        (time, latitude, longitude) float32 ...
    mean_wind_speed_100m       (time, latitude, longitude) float32 ...
    wind_direction_angle_10m   (time, latitude, longitude) float32 ...
    wind_direction_angle_100m  (time, latitude, longitude) float32 ...

#### How to get mean for a data var for every hour in time dimension?
- Use `xarray.DataArray.mean`: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.mean.html

-----

## Taking the mean of my 7 data vars across all lats and lons and assigning to pandas dataframes
- Also localise ERA5 tz naive datetimeindex objects to tz UTC aware!

In [1314]:
# I validated that it is indeed the hourly mean across all lats and lons!
# (ds.sel(time=['2018-01-01 00:00:00'])['mean_wind_speed_10m'].values).mean()

#### All dataframes have index set to datetimeindex

In [1315]:
# Calculate mean across all lats and lons, assign to new df, and convert naive datetimeindex to UTC tz aware

# Units: m/s
df_mean_wind_speed_10m = ds['mean_wind_speed_10m'].mean(dim=['latitude', 'longitude']).to_dataframe()
df_mean_wind_speed_10m.index = df_mean_wind_speed_10m.index.tz_localize(tz='utc')

# Units: m/s
df_mean_wind_speed_100m = ds['mean_wind_speed_100m'].mean(dim=['latitude', 'longitude']).to_dataframe()
df_mean_wind_speed_100m.index = df_mean_wind_speed_100m.index.tz_localize(tz='utc')

# Units: degrees
df_wind_direction_angle_10m = ds['wind_direction_angle_10m'].mean(dim=['latitude', 'longitude']).to_dataframe()
df_wind_direction_angle_10m.index = df_wind_direction_angle_10m.index.tz_localize(tz='utc')

# Units: degrees
df_wind_direction_angle_100m = ds['wind_direction_angle_100m'].mean(dim=['latitude', 'longitude']).to_dataframe()
df_wind_direction_angle_100m.index = df_wind_direction_angle_100m.index.tz_localize(tz='utc')

# Units: Pascals (Pa)
df_msl = ds['msl'].mean(dim=['latitude', 'longitude']).to_dataframe()
df_msl.index = df_msl.index.tz_localize(tz='utc')

# instantaneous_10m_wind_gust; units: m/s
df_i10fg = ds['i10fg'].mean(dim=['latitude', 'longitude']).to_dataframe()
df_i10fg.index = df_i10fg.index.tz_localize(tz='utc')

# 2m_temperature; units: Kelvin
df_t2m = ds['t2m'].mean(dim=['latitude', 'longitude']).to_dataframe()
df_t2m.index = df_t2m.index.tz_localize(tz='utc')

#### Left merge above 7 dataframes of mean values onto `df_main` using indices of both

In [1316]:
df_main = df_main.merge(df_mean_wind_speed_10m, left_index=True, right_index=True, how='left')
df_main = df_main.merge(df_mean_wind_speed_100m, left_index=True, right_index=True, how='left')
df_main = df_main.merge(df_wind_direction_angle_10m, left_index=True, right_index=True, how='left')
df_main = df_main.merge(df_wind_direction_angle_100m, left_index=True, right_index=True, how='left')
df_main = df_main.merge(df_msl, left_index=True, right_index=True, how='left')
df_main = df_main.merge(df_i10fg, left_index=True, right_index=True, how='left')
df_main = df_main.merge(df_t2m, left_index=True, right_index=True, how='left')

In [1317]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49656 entries, 2018-01-01 00:00:00+00:00 to 2023-08-31 23:00:00+00:00
Data columns (total 20 columns):
 #   Column                                          Non-Null Count  Dtype              
---  ------                                          --------------  -----              
 0   hour                                            49656 non-null  int32              
 1   day_of_week                                     49656 non-null  int32              
 2   day_of_month                                    49656 non-null  int32              
 3   month_number                                    49656 non-null  int32              
 4   year                                            49656 non-null  int32              
 5   meteorological_season                           49656 non-null  object             
 6   turbines_in_operation                           49656 non-null  int64              
 7   total_nominal_capacity_operational_tur

In [1318]:
df_main.isna().sum()

hour                                                 0
day_of_week                                          0
day_of_month                                         0
month_number                                         0
year                                                 0
meteorological_season                                0
turbines_in_operation                                0
total_nominal_capacity_operational_turbines_mw       0
datetime_cet                                         2
actual_generated_smard_mwh                           2
day_ahead_price_eur_mwh                           6552
total_nominal_capacity_smard_mw                      2
forecasted_generation_smard_mwh                     26
mean_wind_speed_10m                               1488
mean_wind_speed_100m                              1488
wind_direction_angle_10m                          1488
wind_direction_angle_100m                         1488
msl                                               1488
i10fg     

#### Here I'll rename some features, convert to other units, and derive the intercardinal wind directions!
- Pascals to hectopascal (hPa); same as millibar --> (1 hPa = 1 mb)

In [1319]:
df_main.rename(columns={
    'msl': 'mean_sea_level_pressure_mb',
    'i10fg': 'wind_gusts_10m',
    't2m': 'temp_2m_celsius',
}, inplace=True)

In [ ]:
# Convert Pascals to hPA

In [1321]:
df_main['mean_sea_level_pressure_mb'] = df_main['mean_sea_level_pressure_mb'].apply(lambda pressure: pressure / 100)

In [1322]:
df_main['mean_sea_level_pressure_mb'].mean()

1016.349828267081

In [1323]:
df_main['temp_2m_celsius'] = df_main['temp_2m_celsius'].apply(lambda temp: temp - 273.15)
df_main['temp_2m_celsius'].mean()

10.145321759938463

In [1325]:
df_main.sample(10)

,hour,day_of_week,day_of_month,month_number,year,meteorological_season,turbines_in_operation,total_nominal_capacity_operational_turbines_mw,datetime_cet,actual_generated_smard_mwh,day_ahead_price_eur_mwh,total_nominal_capacity_smard_mw,forecasted_generation_smard_mwh,mean_wind_speed_10m,mean_wind_speed_100m,wind_direction_angle_10m,wind_direction_angle_100m,mean_sea_level_pressure_mb,wind_gusts_10m,temp_2m_celsius
datetime_utc,,,,,,,,,,,,,,,,,,,,
2022-09-06 04:00:00+00:00,4,1,6,9,2022,autumn,29158,57103.802887,2022-09-06 06:00:00+02:00,12800.25,513.03,55289.0,13596.75,3.391586,5.897448,126.921661,137.785858,1019.400078,6.139877,15.372797
2021-01-08 18:00:00+00:00,18,4,8,1,2021,winter,28730,54239.650077,2021-01-08 19:00:00+01:00,4198.25,88.05,54499.0,4038.25,2.582330,4.718844,253.352554,262.573181,1016.701563,4.827219,0.017297
2022-10-04 07:00:00+00:00,7,1,4,10,2022,autumn,29186,57277.306087,2022-10-04 09:00:00+02:00,4860.75,317.09,55289.0,4911.50,2.892068,4.617548,207.119766,215.766357,1022.545234,5.509134,9.227411
2022-04-29 09:00:00+00:00,9,4,29,4,2022,spring,29004,56298.211787,2022-04-29 11:00:00+02:00,1170.25,217.40,55289.0,1233.00,1.965918,2.477643,140.997635,140.611710,1028.399297,5.378484,12.333459
2019-02-01 00:00:00+00:00,0,4,1,2,2019,winter,28280,52251.896317,2019-02-01 01:00:00+01:00,16768.00,45.47,52792.0,15331.25,4.022220,7.263328,120.264992,126.338173,996.369922,7.314440,-1.267187
2019-06-09 12:00:00+00:00,12,6,9,6,2019,summer,28306,52371.040417,2019-06-09 14:00:00+02:00,2190.50,14.93,52792.0,2586.25,2.771303,3.676010,159.370041,160.170334,1022.257500,7.068283,20.791162
2019-08-29 10:00:00+00:00,10,3,29,8,2019,summer,28332,52498.971817,2019-08-29 12:00:00+02:00,5198.00,42.12,52792.0,4020.75,2.622577,3.403124,252.773987,251.450089,1018.399844,6.996563,23.476495
2020-07-12 17:00:00+00:00,17,6,12,7,2020,summer,28631,53609.552667,2020-07-12 19:00:00+02:00,3161.50,35.12,53184.0,4025.50,3.444699,4.751866,169.160248,169.217422,1025.764453,6.419269,19.424554
2022-02-09 21:00:00+00:00,21,2,9,2,2022,winter,28976,55990.984347,2022-02-09 22:00:00+01:00,11560.00,211.79,55289.0,11362.75,3.491689,6.307554,227.475677,238.621002,1023.761172,6.256780,5.405145


#### Save `df_main` progress to pickle

In [1326]:
# df_main.to_pickle(data_push_path / 'df_main_smard_era5.pkl')

# Next step: derive intercardinal directions!

# Not so fast! The means calculated above are over ALL lats and lons in the xarray dataset. This is NOT the unique c. 700 grid points!
### Fix this!